In [1]:
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
# !sudo apt-get install git-lfs
# !git lfs install

# #!rm -rf anomaly-detection-datasets

# !git clone https://github.com/kotlarmilos/anomaly-detection-datasets.git 
# !cd anomaly-detection-datasets && git pull
# !ls -la anomaly-detection-datasets
# !pip install pymfe
# !ls -la anomaly-detection-datasets

python -m pip install mysql-connector

In [ ]:


# data cleansing
import numpy as np
import pandas as pd
from pymfe.mfe import MFE
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def data_transformation(uri, target=None, mfe=True, nrows=1000):
    df = pd.read_csv(uri,
                     dtype={'srcip': 'object','dstip': 'object', 'ct_dst_src_ltm': 'int64'}, 
                     nrows=nrows)

    # convert objects to category codes
    for col_name in df.columns:
        if(df[col_name].dtype == 'object'):
            df[col_name]= df[col_name].astype('category')
            df[col_name] = df[col_name].cat.codes.astype('int64')

    # split dataset into features and target arrays
    dataset = df.to_numpy()
    features = np.nan_to_num(df.drop(columns=['Label']).to_numpy())
    target = df['Label'].to_numpy()

    # Extract general, statistical and information-theoretic measures
    ft = ()
    if (mfe==True):
        mfe = MFE(groups=["general", "statistical", "info-theory"])
        mfe.fit(features, target)
        ft = mfe.extract()
    
    return dataset, features, target, ft

In [3]:
#initialize tensorflow
import tensorflow as tf
from tensorflow.python.client import device_lib
import os
strategy = None
if (os.environ.get('COLAB_TPU_ADDR')!=None):
  os.environ['TPU_ADDR'] = os.environ['COLAB_TPU_ADDR']
# else:
#   os.environ['TPU_ADDR'] = '10.15.20.26:8470'

if (os.environ.get('TPU_ADDR')!=None):
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.67.69.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.67.69.90:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
def kNN(features_tf, target_tf, sample_tf, k_tf):
    neg_one = tf.constant(-1.0, dtype=tf.float64)
    # we compute the L-1 distance
    distances =  tf.reduce_sum(tf.abs(tf.subtract(features_tf, sample_tf)), 1)
    # to find the nearest points, we find the farthest points based on negative distances
    # we need this trick because tensorflow has top_k api and no closest_k or reverse=True api
    neg_distances = tf.multiply(distances, neg_one)
    # get the indices
    vals, indx = tf.nn.top_k(neg_distances, k_tf)
    # slice the labels of these points
    ps = tf.gather(target_tf, indx)
    counts = np.bincount(ps.numpy().astype('int64'))
    return np.argmax(counts)

In [ ]:
def run_on_device(device_name, method, parameters, features, target=None, n_folds=None):
    mean_exec_time = mean_latency = tn_mean = fp_mean = fn_mean = tp_mean = logloss_mean = auc_mean = abs_err_mean = sq_err_mean =  []
    i=1
    k_tf = tf.constant(parameters['k'])
    print('====== Model details ======')
    print('Method:', method)
    print('Parameters:', parameters)
    print('Dataset size (instances, features):', features.shape)
    print('Number of folds:', n_folds)
    print('Train instances per fold:', features.shape[0] - features.shape[0]/n_folds)
    print('Test instances per fold:', features.shape[0]/n_folds)
    print('===========================')
    print()
    print('====== Hardware details ======')
    print('Device name:', device_name)
    print('Tensorflow version:', tf.__version__)
    print('===========================')
    print()
    print('====== Execution details ======')
    kf = KFold(n_splits=n_folds, shuffle=True)
    for train_index, test_index in kf.split(features):
        features_train, features_test, target_train, target_test = features[train_index], features[test_index], target[train_index], target[test_index]
        features_tf = tf.constant(features_train)
        target_tf = tf.constant(target_train)
        scores = []
        
        start_time = tf.timestamp()
        for sample in features_test:
            sample_tf = tf.constant(sample)
            p = kNN(features_tf, target_tf, sample_tf, k_tf)
            scores = np.append(scores,p)
        elapsed_time = tf.timestamp()-start_time
        tn, fp, fn, tp = confusion_matrix(target_test, scores,labels=[0,1]).ravel()
        logloss_mean= np.append(logloss_mean,log_loss(target_test, scores, eps=1e-15,labels=[0,1]))
        try:
            auc_mean= np.append(auc_mean,roc_auc_score(target_test, scores))
        except:
            auc_mean = 0
        abs_err_mean = np.append(abs_err_mean, mean_absolute_error(target_test, scores))
        sq_err_mean = np.append(sq_err_mean, mean_squared_error(target_test, scores))
        tn_mean= np.append(tn_mean,tn)
        fp_mean= np.append(fp_mean,fp)
        fn_mean= np.append(fn_mean,fn)
        tp_mean= np.append(tp_mean,tp)
        mean_exec_time.append(elapsed_time)
        mean_latency.append(elapsed_time/target_test.size)
        print(str(i)+". Fold (execution time:",str(elapsed_time.numpy()),"seconds, latency:",elapsed_time.numpy()/scores.size)
        i+=1
    
    tn, fp, fn, tp, logloss, auc, sq_mean_err, abs_mean_err = np.mean(tn_mean),np.mean(fp_mean),np.mean(fn_mean),np.mean(tp_mean),np.mean(logloss_mean), np.mean(auc_mean), np.mean(sq_err_mean),np.mean(abs_err_mean)
    total_examples = target.size/n_folds
    acc = (tp+tn)/total_examples
    prec = (tp)/(tp+fp)
    recall = (tp)/(tp+fn)
    f1 = 2*(prec*recall)/(prec+recall)
    print()
    print('====== Summary details ======')
    print('Accuracy:', acc*100)
    print('Precision:', prec*100)
    print('Recall:', recall*100)
    print('F1 score:', f1*100)
    print('Log loss:',logloss)
    print('AUC:',auc)
    print('Mean squared error:',sq_mean_err)
    print('Mean abs error:',abs_mean_err)
    print()
    print("Confusion matrix","negative (predicted)  positive (predicted)")
    print("negative (actual)       ", tn,"              ",fp)
    print("positive (actual)       ", fn,"              ",tp)
    print()
    print('Mean execution time:', np.mean(mean_exec_time))
    print('Mean latency:', np.mean(mean_latency))
    

In [ ]:
def build_model(method, parameters, features, target=None, n_folds=None):
    local_device_protos = device_lib.list_local_devices()
    for device in local_device_protos:
        if ('XLA' not in device.name):
            with tf.device(device.name):
                run_on_device(device.name, method, parameters, features, target, n_folds)
    if (strategy):
        with strategy.scope():
            run_on_device('TPU', method, parameters, features, target, n_folds)

        

In [7]:

    
dataset, features, target, ft = data_transformation('./anomaly-detection-datasets/UNSW-NB15_1.csv', 'Label', True, 10000);
build_model('kNN',{'k':5}, features, target, 10)

====== Model details ======
Method: kNN
Parameters: {'k': 5}
Dataset size (instances, features): (10000, 48)
Number of folds: 10
Train instances per fold: 9000.0
Test instances per fold: 1000.0

====== Hardware details ======
Device name: /device:CPU:0
Tensorflow version: 2.2.0-rc4

====== Execution details ======
1. Fold (execution time: 3.1973419189453125 seconds, latency: 0.0031973419189453125
2. Fold (execution time: 2.9479610919952393 seconds, latency: 0.0029479610919952394
3. Fold (execution time: 2.8650691509246826 seconds, latency: 0.0028650691509246825
4. Fold (execution time: 2.9485721588134766 seconds, latency: 0.0029485721588134767
5. Fold (execution time: 3.255005121231079 seconds, latency: 0.003255005121231079
6. Fold (execution time: 3.148874044418335 seconds, latency: 0.0031488740444183348
7. Fold (execution time: 2.824737071990967 seconds, latency: 0.002824737071990967
8. Fold (execution time: 3.0110161304473877 seconds, latency: 0.0030110161304473877
9. Fold (executio

In [8]:
print(ft)

(['attr_conc.mean', 'attr_conc.sd', 'attr_ent.mean', 'attr_ent.sd', 'attr_to_inst', 'can_cor.mean', 'can_cor.sd', 'cat_to_num', 'class_conc.mean', 'class_conc.sd', 'class_ent', 'cor.mean', 'cor.sd', 'cov.mean', 'cov.sd', 'eigenvalues.mean', 'eigenvalues.sd', 'eq_num_attr', 'freq_class.mean', 'freq_class.sd', 'g_mean.mean', 'g_mean.sd', 'gravity', 'h_mean.mean', 'h_mean.sd', 'inst_to_attr', 'iq_range.mean', 'iq_range.sd', 'joint_ent.mean', 'joint_ent.sd', 'kurtosis.mean', 'kurtosis.sd', 'mad.mean', 'mad.sd', 'max.mean', 'max.sd', 'mean.mean', 'mean.sd', 'median.mean', 'median.sd', 'min.mean', 'min.sd', 'mut_inf.mean', 'mut_inf.sd', 'nr_attr', 'nr_bin', 'nr_cat', 'nr_class', 'nr_cor_attr', 'nr_disc', 'nr_inst', 'nr_norm', 'nr_num', 'nr_outliers', 'ns_ratio', 'num_to_cat', 'range.mean', 'range.sd', 'sd.mean', 'sd.sd', 'sd_ratio', 'skewness.mean', 'skewness.sd', 'sparsity.mean', 'sparsity.sd', 't_mean.mean', 't_mean.sd', 'var.mean', 'var.sd', 'w_lambda'], [0.13965335, 0.21842285, 2.6470037